In [1]:
# Import data
import numpy as np
import pandas as pd
from functions import *
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
import statsmodels.api as sm 
import sqlite3
import os

X_used, Y = get_data(42)
n = X_used.shape[0]

grid_size = 50
a_grid = np.linspace(np.min(X_used.age), np.max(X_used.age), grid_size)
w_grid = np.linspace(np.min(X_used.hh_inc), np.max(X_used.hh_inc), grid_size)
xv, wv = np.meshgrid(a_grid, w_grid, indexing='ij')

X_used_cnst = sm.add_constant(X_used)
model = sm.OLS(Y,X_used_cnst)
results = model.fit()
# results.summary()

In [2]:
# Make Database
database_name = 'database_happiness_PL.db'
con = sqlite3.connect(os.path.join('Results', database_name))
cur = con.cursor()

res = cur.execute("""SELECT name FROM sqlite_master WHERE type='table'""")
table_names = res.fetchall()
if ~np.isin('PL', table_names):
    print("CREATE NEW DATABASE TABLE")
    cur.execute("""CREATE TABLE IF NOT EXISTS PL(
                Method TEXT NOT NULL,
                Model TEXT NOT NULL,
                Parameter_v TEXT NOT NULL,
                Parameter_y TEXT NOT NULL,
                Seed INTEGER NOT NULL,
                Treatment TEXT NOT NULL,
                Control_1 TEXT NOT NULL,
                Control_2 TEXT NOT NULL,
                Value REAL NOT NULL,
                Con_val_1 REAL NOT NULL,
                Con_val_2 REAL NOT NULL,
                PRIMARY KEY (Method, Model, Seed, Treatment, Control_1, Control_2))""")
    con.commit()
else:
    print("DATABASE TABLE ALREADY EXISTS")


DATABASE TABLE ALREADY EXISTS


In [3]:
h = 1  # Bandwidth parameter
weights = np.zeros((n, grid_size))
for i in range(grid_size):
    u = np.abs(X_used['hh_inc'].values - w_grid[i])/h
    val = Gaussian(u)
    weights[:,i] = val/np.sum(val)

characteristic_names = ['zerotofive','sixtotwenty','grp','home_own','gender','isHHH',
                        'live_together','work','get_social_benefit',
                        'got_social_benefit','religion','marriage','health',
                        'exercise','smoke','alcohol']

ATE_names = ['zerotofive','sixtotwenty','grp','home_own','gender','isHHH',
                        'live_together','work','get_social_benefit',
                        'got_social_benefit','religion','marriage','health',
                        'exercise','smoke','alcohol']

plot_names = ['Baby - No Baby', 'Teenager - No Teenager', 'Grandparents - No Grandparents','Homeowner - Lease',
              'Female - Male', 'Head - Not Head', 'Live Together - Separate', 
              'Work - No Work','Receive Social Insurance - Has Never Received', 'Had Received Social Insurance - Has Never Received',
              'Religion - No Religion','Married - Not Married', 'Health',
              'Exercise', 'Smoke - No Smoke', 'Alcohol - No Alcohol']

n_jobs = 10
# seed_list = [42, 43, 44, 45, 46]
seed_list = [44, 45, 46]
CV_grid_n = 10

In [4]:
############################################################################
######################## Debiased ML, Random Forest ########################
############################################################################
max_depth_list = np.append(np.linspace(1,50,CV_grid_n-1).astype(int),None)
err = 0
PL_RF = {}
PL_RF['Method'] = ['PL']*len(seed_list)*len(characteristic_names)
PL_RF['Model'] = ['RF']*len(seed_list)*len(characteristic_names)
PL_RF['Parameter_v'] = []
PL_RF['Parameter_y'] = []
PL_RF['seed'] = []
PL_RF['Treatment'] = []
PL_RF['Control_1'] = ['hh_inc']*len(seed_list)*len(characteristic_names)
PL_RF['Control_2'] = ['age']*len(seed_list)*len(characteristic_names)
PL_RF['Value'] = []
PL_RF['Con_val_1'] = []
PL_RF['Con_val_2'] = []

K = 2

for seed in seed_list:
    divide_idx = [int(i) for i in np.linspace(0,n,K+1)]
    for j, characteristic in enumerate(characteristic_names):
        print(characteristic)
        PL_RF['seed'].append(seed)
        PL_RF['Treatment'].append(characteristic)
        V_used = X_used[np.append(characteristic,['hh_inc', 'age'])]
        W_used = X_used.drop(np.append(characteristic,['hh_inc', 'age']),axis=1)
        Y_resi = np.zeros((n,))
        V_resi = np.zeros((n,V_used.shape[1]))
        for i in range(K):
            V_linear = V_used.iloc[divide_idx[i]:divide_idx[i+1]]
            W_linear = W_used.iloc[divide_idx[i]:divide_idx[i+1],:]
            Y_linear = Y.iloc[divide_idx[i]:divide_idx[i+1]]

            V_nonlinear = V_used.drop(index=V_used.iloc[divide_idx[i]:divide_idx[i+1]].index,axis=0)
            W_nonlinear = W_used.drop(index=W_used.iloc[divide_idx[i]:divide_idx[i+1]].index,axis=0)
            Y_nonlinear = Y.drop(index=Y.iloc[divide_idx[i]:divide_idx[i+1]].index,axis=0)

            if i == 0:
                CV_error_RF = np.zeros((len(max_depth_list),))
                for cv_i, max_depth in enumerate(max_depth_list):
                    clf = RandomForestRegressor(n_estimators=100, max_depth=max_depth, random_state=seed, n_jobs=n_jobs)
                    scores = cross_val_score(clf, W_nonlinear, V_nonlinear, cv=5, scoring='neg_mean_squared_error')
                    CV_error_RF[cv_i] = np.mean(scores)
                # if CV_error_RF.argmax()==len(max_depth_list)-2:
                #     err = err+1
                #     raise ValueError('Tuning parameter at boundary')
                max_depth_v = max_depth_list[CV_error_RF.argmax()]
                print(max_depth_v)
                
            RF_model = RandomForestRegressor(n_estimators=100, max_depth=max_depth_v, random_state=seed, n_jobs=n_jobs).fit(W_nonlinear, V_nonlinear)

            V_linear_resi = V_linear - RF_model.predict(W_linear)

            if i == 0:
                CV_error_RF = np.zeros((len(max_depth_list),))
                for cv_i, max_depth in enumerate(max_depth_list):
                    clf = RandomForestRegressor(n_estimators=100, max_depth=max_depth, random_state=seed, n_jobs=n_jobs)
                    scores = cross_val_score(clf, W_nonlinear, Y_nonlinear, cv=5, scoring='neg_mean_squared_error')
                    CV_error_RF[cv_i]=np.mean(scores)
                # if CV_error_RF.argmax()==len(max_depth_list)-2:
                #     err = err+1
                #     raise ValueError('Tuning parameter at boundary')
                max_depth_y = max_depth_list[CV_error_RF.argmax()]
                print(max_depth_y)
                
            RF_model = RandomForestRegressor(n_estimators=100, max_depth=max_depth_y, random_state=seed, n_jobs=n_jobs).fit(W_nonlinear, Y_nonlinear)
            
            Y_linear_resi = Y_linear - RF_model.predict(W_linear)
            
            V_resi[divide_idx[i]:divide_idx[i+1]] = V_linear_resi.values
            Y_resi[divide_idx[i]:divide_idx[i+1]] = Y_linear_resi.values
        
        if max_depth_v==None:
            max_depth_v='None'        
        PL_RF['Parameter_v'].append(max_depth_v)
        if max_depth_y==None:
            max_depth_y='None'        
        PL_RF['Parameter_y'].append(max_depth_y)

        beta_hat = np.linalg.inv(V_resi.T @ V_resi) @ (V_resi.T @ Y_resi)
        PL_RF['Value'].append(beta_hat[0])
        PL_RF['Con_val_1'].append(beta_hat[1])
        PL_RF['Con_val_2'].append(beta_hat[2])


PL_RF = pd.DataFrame(PL_RF)
out = PL_RF.values
query = ''' insert or replace into PL (Method,Model,Parameter_v,Parameter_y,seed,Treatment,Control_1,Control_2,Value,Con_val_1,Con_val_2) values (?,?,?,?,?,?,?,?,?,?,?) '''
cur.executemany(query, out)
con.commit()

zerotofive
13
7
sixtotwenty
13
7
grp
13
7
home_own
13
7
gender
13
7
isHHH
13
7
live_together
13
7
work
13
7
get_social_benefit
13
7
got_social_benefit
13
7
religion
13
7
marriage
13
7
health
13
7
exercise
13
7
smoke
13
7
alcohol
13
7
zerotofive
13
7
sixtotwenty
13
7
grp
13
7
home_own
13
7
gender
13
7
isHHH
13
7
live_together
13
7
work
13
7
get_social_benefit
13
7
got_social_benefit
13
7
religion
13
7
marriage
13
7
health
13
7
exercise
13
7
smoke
13
7
alcohol
13
7
zerotofive
13
7
sixtotwenty
13
7
grp
13
7
home_own
13
7
gender
13
7
isHHH
13
7
live_together
13
7
work
13
7
get_social_benefit
13
7
got_social_benefit
13
7
religion
13
7
marriage
13
7
health
13
7
exercise
13
7
smoke
13
7
alcohol
13
7


In [5]:

############################################################################
########################### Debiased ML, XGB   #############################
############################################################################
n_estimators_list = np.linspace(1,150,CV_grid_n).astype(int)

err = 0
PL_XGB = {}
PL_XGB['Method'] = ['PL']*len(seed_list)*len(characteristic_names)
PL_XGB['Model'] = ['XGB']*len(seed_list)*len(characteristic_names)
PL_XGB['Parameter_v'] = []
PL_XGB['Parameter_y'] = []
PL_XGB['seed'] = []
PL_XGB['Treatment'] = []
PL_XGB['Control_1'] = ['hh_inc']*len(seed_list)*len(characteristic_names)
PL_XGB['Control_2'] = ['age']*len(seed_list)*len(characteristic_names)
PL_XGB['Value'] = []
PL_XGB['Con_val_1'] = []
PL_XGB['Con_val_2'] = []

K = 2

for seed in seed_list:
    divide_idx = [int(i) for i in np.linspace(0,n,K+1)]
    for j, characteristic in enumerate(characteristic_names):
        print(characteristic)
        PL_XGB['seed'].append(seed)
        PL_XGB['Treatment'].append(characteristic)
        V_used = X_used[np.append(characteristic,['hh_inc', 'age'])]
        W_used = X_used.drop(np.append(characteristic,['hh_inc', 'age']),axis=1)
        Y_resi = np.zeros((n,))
        V_resi = np.zeros((n,V_used.shape[1]))
        for i in range(K):
            V_linear = V_used.iloc[divide_idx[i]:divide_idx[i+1]]
            W_linear = W_used.iloc[divide_idx[i]:divide_idx[i+1],:]
            Y_linear = Y.iloc[divide_idx[i]:divide_idx[i+1]]

            V_nonlinear = V_used.drop(index=V_used.iloc[divide_idx[i]:divide_idx[i+1]].index,axis=0)
            W_nonlinear = W_used.drop(index=W_used.iloc[divide_idx[i]:divide_idx[i+1]].index,axis=0)
            Y_nonlinear = Y.drop(index=Y.iloc[divide_idx[i]:divide_idx[i+1]].index,axis=0)

            if i == 0:
                CV_error_XGB = np.zeros((len(n_estimators_list),))
                for n_idx,n_estimators in enumerate(n_estimators_list):

                    XGB_model = xgb.XGBRegressor(n_jobs=n_jobs, tree_method="exact", n_estimators=n_estimators,
                                random_state=seed)
                    scores = cross_val_score(XGB_model, W_nonlinear, V_nonlinear, cv=5, scoring='neg_mean_squared_error')
                    CV_error_XGB[n_idx] = np.mean(scores)
                print(n_estimators_list[CV_error_XGB.argmax()])
                n_estimators_v = n_estimators_list[CV_error_XGB.argmax()]

            XGB_model = xgb.XGBRegressor(n_jobs=n_jobs, tree_method="exact", n_estimators=n_estimators_v, random_state=seed).fit(W_nonlinear, V_nonlinear)
            V_linear_resi = V_linear - XGB_model.predict(W_linear)

            if i == 0:
                CV_error_XGB = np.zeros((len(n_estimators_list),))
                for n_idx,n_estimators in enumerate(n_estimators_list):
                    XGB_model = xgb.XGBRegressor(n_jobs=n_jobs, tree_method="exact", n_estimators=n_estimators,
                                random_state=seed)
                    scores = cross_val_score(XGB_model, W_nonlinear, Y_nonlinear, cv=5, scoring='neg_mean_squared_error')
                    CV_error_XGB[n_idx] = np.mean(scores)
                # if CV_error_XGB.argmax()==n_est_max-1:
                #     err = err+1
                #     raise ValueError('Tuning parameter at boundary')
                print(n_estimators_list[CV_error_XGB.argmax()])
                n_estimators_y = n_estimators_list[CV_error_XGB.argmax()]
                
            XGB_model = xgb.XGBRegressor(n_jobs=n_jobs, tree_method="exact", n_estimators=n_estimators_y,
                                random_state=seed).fit(W_nonlinear, Y_nonlinear)
            
            Y_linear_resi = Y_linear - XGB_model.predict(W_linear)
            
            V_resi[divide_idx[i]:divide_idx[i+1]] = V_linear_resi.values
            Y_resi[divide_idx[i]:divide_idx[i+1]] = Y_linear_resi.values
        
        PL_XGB['Parameter_v'].append(n_estimators_v)
        PL_XGB['Parameter_y'].append(n_estimators_y)

        beta_hat = np.linalg.inv(V_resi.T @ V_resi) @ (V_resi.T @ Y_resi)
        PL_XGB['Value'].append(beta_hat[0])
        PL_XGB['Con_val_1'].append(beta_hat[1])
        PL_XGB['Con_val_2'].append(beta_hat[2])


PL_XGB = pd.DataFrame(PL_XGB)
out = PL_XGB.values
query = ''' insert or replace into PL (Method,Model,Parameter_v,Parameter_y,seed,Treatment,Control_1,Control_2,Value,Con_val_1,Con_val_2) values (?,?,?,?,?,?,?,?,?,?,?) '''
cur.executemany(query, out)
con.commit()

zerotofive
34
17
sixtotwenty
34
17
grp
34
17
home_own
17
17
gender
17
17
isHHH
34
17
live_together
17
17
work
17
17
get_social_benefit
17
17
got_social_benefit
34
17
religion
34
17
marriage
34
17
health
17
17
exercise
17
17
smoke
34
17
alcohol
34
17
zerotofive
34
17
sixtotwenty
34
17
grp
34
17
home_own
17
17
gender
17
17
isHHH
34
17
live_together
17
17
work
17
17
get_social_benefit
17
17
got_social_benefit
34
17
religion
34
17
marriage
34
17
health
17
17
exercise
17
17
smoke
34
17
alcohol
34
17
zerotofive
34
17
sixtotwenty
34
17
grp
34
17
home_own
17
17
gender
17
17
isHHH
34
17
live_together
17
17
work
17
17
get_social_benefit
17
17
got_social_benefit
34
17
religion
34
17
marriage
34
17
health
17
17
exercise
17
17
smoke
34
17
alcohol
34
17


In [ ]:

############################################################################
########################### Debiased ML, XGBs   #############################
############################################################################

n_estimators_list = np.linspace(1,50,CV_grid_n).astype(int)
num_parallel_tree = 100
subsample = np.sqrt(X_used.shape[0])/X_used.shape[0]

err = 0
PL_XGBs = {}
PL_XGBs['Method'] = ['PL']*len(seed_list)*len(characteristic_names)
PL_XGBs['Model'] = ['XGBs']*len(seed_list)*len(characteristic_names)
PL_XGBs['Parameter_v'] = []
PL_XGBs['Parameter_y'] = []
PL_XGBs['seed'] = []
PL_XGBs['Treatment'] = []
PL_XGBs['Control_1'] = ['hh_inc']*len(seed_list)*len(characteristic_names)
PL_XGBs['Control_2'] = ['age']*len(seed_list)*len(characteristic_names)
PL_XGBs['Value'] = []
PL_XGBs['Con_val_1'] = []
PL_XGBs['Con_val_2'] = []

K = 2

for seed in seed_list:
    divide_idx = [int(i) for i in np.linspace(0,n,K+1)]
    for j, characteristic in enumerate(characteristic_names):
        print(characteristic)
        PL_XGBs['seed'].append(seed)
        PL_XGBs['Treatment'].append(characteristic)
        V_used = X_used[np.append(characteristic,['hh_inc', 'age'])]
        W_used = X_used.drop(np.append(characteristic,['hh_inc', 'age']),axis=1)
        Y_resi = np.zeros((n,))
        V_resi = np.zeros((n,V_used.shape[1]))
        for i in range(K):
            V_linear = V_used.iloc[divide_idx[i]:divide_idx[i+1]]
            W_linear = W_used.iloc[divide_idx[i]:divide_idx[i+1],:]
            Y_linear = Y.iloc[divide_idx[i]:divide_idx[i+1]]

            V_nonlinear = V_used.drop(index=V_used.iloc[divide_idx[i]:divide_idx[i+1]].index,axis=0)
            W_nonlinear = W_used.drop(index=W_used.iloc[divide_idx[i]:divide_idx[i+1]].index,axis=0)
            Y_nonlinear = Y.drop(index=Y.iloc[divide_idx[i]:divide_idx[i+1]].index,axis=0)

            if i == 0:
                CV_error_XGBs = np.zeros((len(n_estimators_list),))
                for n_idx,n_estimators in enumerate(n_estimators_list):

                    XGBs_model = xgb.XGBRegressor(n_jobs=n_jobs, tree_method="exact", n_estimators=n_estimators,
                                                  num_parallel_tree = num_parallel_tree, subsample = subsample, random_state=seed)
                    scores = cross_val_score(XGBs_model, W_nonlinear, V_nonlinear, cv=5, scoring='neg_mean_squared_error')
                    CV_error_XGBs[n_idx] = np.mean(scores)
                print(n_estimators_list[CV_error_XGBs.argmax()])
                n_estimators_v = n_estimators_list[CV_error_XGBs.argmax()]

            XGBs_model = xgb.XGBRegressor(n_jobs=n_jobs, tree_method="exact", n_estimators=n_estimators_v,
                                          num_parallel_tree = num_parallel_tree, subsample = subsample, random_state=seed).fit(W_nonlinear, V_nonlinear)
            V_linear_resi = V_linear - XGBs_model.predict(W_linear)

            if i == 0:
                CV_error_XGBs = np.zeros((len(n_estimators_list),))
                for n_idx,n_estimators in enumerate(n_estimators_list):
                    XGBs_model = xgb.XGBRegressor(n_jobs=n_jobs, tree_method="exact", n_estimators=n_estimators,
                                                  num_parallel_tree = num_parallel_tree, subsample = subsample, 
                                random_state=seed)
                    scores = cross_val_score(XGBs_model, W_nonlinear, Y_nonlinear, cv=5, scoring='neg_mean_squared_error')
                    CV_error_XGBs[n_idx] = np.mean(scores)
                # if CV_error_XGBs.argmax()==n_est_max-1:
                #     err = err+1
                #     raise ValueError('Tuning parameter at boundary')
                print(n_estimators_list[CV_error_XGBs.argmax()])
                n_estimators_y = n_estimators_list[CV_error_XGBs.argmax()]
                
            XGBs_model = xgb.XGBRegressor(n_jobs=n_jobs, tree_method="exact", n_estimators=n_estimators_y,
                                          num_parallel_tree = num_parallel_tree, subsample = subsample, 
                                random_state=seed).fit(W_nonlinear, Y_nonlinear)
            
            Y_linear_resi = Y_linear - XGBs_model.predict(W_linear)
            
            V_resi[divide_idx[i]:divide_idx[i+1]] = V_linear_resi.values
            Y_resi[divide_idx[i]:divide_idx[i+1]] = Y_linear_resi.values
        
        PL_XGBs['Parameter_v'].append(n_estimators_v)
        PL_XGBs['Parameter_y'].append(n_estimators_y)

        beta_hat = np.linalg.inv(V_resi.T @ V_resi) @ (V_resi.T @ Y_resi)
        PL_XGBs['Value'].append(beta_hat[0])
        PL_XGBs['Con_val_1'].append(beta_hat[1])
        PL_XGBs['Con_val_2'].append(beta_hat[2])


PL_XGBs = pd.DataFrame(PL_XGBs)
out = PL_XGBs.values
query = ''' insert or replace into PL (Method,Model,Parameter_v,Parameter_y,seed,Treatment,Control_1,Control_2,Value,Con_val_1,Con_val_2) values (?,?,?,?,?,?,?,?,?,?,?) '''
cur.executemany(query, out)
con.commit()
cur.close()
con.close()

zerotofive
50
28
sixtotwenty
50
28
grp
50
28
home_own
50
28
gender
50
28
isHHH
50
44
live_together
50
44
work
50
28
get_social_benefit
50
28
got_social_benefit
50
28
religion
50
39
marriage
50
28
health
50
39
exercise
50
28
smoke
50
39
alcohol
50
28
zerotofive
50
44
sixtotwenty
50
44
grp
50
39
home_own
50
44
gender
50
39
isHHH
50
39
live_together
50
39
work
50
44
get_social_benefit
50
39
got_social_benefit
50
39
religion
50
39
marriage
50
39
health
50
44
exercise
50
39
smoke
50
44
alcohol
50
44
zerotofive
50
50
sixtotwenty
50
39
grp
50
39
home_own
50
39
gender
50
28
isHHH
